# 合并新的标签数据集

## 功能说明
- **原始数据集**: `a_marked.csv` (包含所有数据)
- **异常数据集**: `副本筛选后数据（带标记规则代码）.csv` (只包含异常数据)
- **逻辑**: 如果原始数据中的记录出现在异常数据集中，则 Matched=TRUE，否则 Matched=FALSE

# 根据异常数据集更新原始数据的 Matched 字段

In [13]:
import pandas as pd
import os

# 文件路径
original_data_path = '../data_new/a_marked.csv'
anomaly_data_path = '../data_new/副本筛选后数据（带标记规则代码）.csv'

print("加载数据文件...")
print(f"原始数据文件: {original_data_path}")
print(f"异常数据文件: {anomaly_data_path}")

# 加载数据
df_all = pd.read_csv(original_data_path)
df_abnormal = pd.read_csv(anomaly_data_path)

# === Step 2: 定义用于匹配的关键字段 ===
# 请根据你的数据实际情况修改以下字段
key_fields = ['CUST_ID','CONF_DATE', 'CONF_SNO']  # 举例，可替换为你实际的主键字段组合

df_all['Matched'] = False  # 初始化匹配列

# === Step 3: 创建唯一标识列，方便匹配 ===
df_all['__key__'] = df_all[key_fields].astype(str).agg('|'.join, axis=1)
df_abnormal['__key__'] = df_abnormal[key_fields].astype(str).agg('|'.join, axis=1)

# === Step 4: 标记匹配情况 ===
df_all['Matched'] = df_all['__key__'].isin(set(df_abnormal['__key__']))

# === Step 5: 删除中间列，保存结果 ===
df_all.drop(columns='__key__', inplace=True)
df_all.to_csv('a_marked_with_matched.csv', index=False)


print('标记完成，已保存为 a_marked_with_matched.csv')

# === Step 6: Matched 字段统计 ===
matched_stats = df_all['Matched'].value_counts().rename_axis('Matched').reset_index(name='Count')
print('\nMatched 字段统计结果：')
print(matched_stats)

print(f"\n异常数据集总记录数：{len(df_abnormal)}")

加载数据文件...
原始数据文件: ../data_new/a_marked.csv
异常数据文件: ../data_new/副本筛选后数据（带标记规则代码）.csv


/tmp/ipykernel_117929/12788415.py:13: DtypeWarning: Columns (7,18,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv(original_data_path)


标记完成，已保存为 a_marked_with_matched.csv

Matched 字段统计结果：
   Matched  Count
0    False  49376
1     True   4696

异常数据集总记录数：4696
